首页日期格式不正确


In [17]:
#首页日期检查
import re
from docx import Document

def is_chinese_date_format(text):
    pattern = r'[〇一二三四五六七八九零]{4}年[〇一二三四五六七八九十]{1,2}月'
    match = re.search(pattern, text)
    return bool(match)

def get_first_page_text(doc):
    first_page_text = ''
    for paragraph in doc.paragraphs:
    
        first_page_text += paragraph.text + '\n'
        # 假设文档是简单结构，遇到换页符后停止
        if 'PAGE_BREAK' in paragraph.text:
            break
    return first_page_text

def check_first_page_date_format(docx_path):
    doc = Document(docx_path)
    first_page_text = get_first_page_text(doc)

    if is_chinese_date_format(first_page_text):
        print("首页日期格式正确")
    else:
        print("首页日期格式不正确")

# 使用该函数检测文档
check_first_page_date_format('test.docx')


首页日期格式正确


In [22]:
# 设计文件分发表的邮箱、地址、手机号码等应准确，我方人员应为合同制员工
import re
import csv
from docx import Document


def extract_person_info(text):
    zong_name_match = re.search(r'项目总负责人：([^\n]+)', text)
    dan_name_match = re.search(r'单项设计负责人：([^\n]+)',text)
    jian_name_match = re.search(r'建设单位联系人：([^\n]+)',text)
    tel_match = re.findall(r'电话：([^\n]+)', text)
    mail_match = re.findall(r'电子邮箱：([^\n]+)', text)

    persons = []
    if zong_name_match and len(tel_match)>0 and len(mail_match)>0:
        person1={
            'name':zong_name_match.group(1).strip(),
            'tel':tel_match[0].strip(),
            'mail':mail_match[0].strip()
        }
        persons.append(person1)
    if dan_name_match and len(tel_match) > 1 and len(mail_match) > 1:
        person2={
            'name':dan_name_match.group(1).strip(),
            'tel':tel_match[1].strip(),
            'mail':mail_match[1].strip()
        }   
        persons.append(person2)
    if jian_name_match and len(tel_match) > 2 and len(mail_match) > 2 :
        person3={
            'name':jian_name_match.group(1).strip(),
            'tel':tel_match[2].strip(),
            'mail':mail_match[2].strip()
        }
        persons.append(person3)
    return persons if persons else None

def find_table_after_paragraph(docx_path, paragraph_text):
    doc = Document(docx_path)
    found_paragraph = False
    for para in doc.paragraphs:
        if paragraph_text in para.text:
            found_paragraph = True
        if found_paragraph:
            for table in doc.tables:
                if para._element.getnext() is table._element:
                    last_row = table.rows[-1]
                    last_cell = last_row.cells[-1]
                    return last_cell.text.strip()

    return None


docx_path = 'test.docx'  # 替换为你的文件路径
paragraph_text = '设计文件分发表'  # 替换为段落中的目标文字

content = find_table_after_paragraph(docx_path, paragraph_text)
persons = extract_person_info(content)
print(persons)


def find_person_in_csv(csv_path, persons):
    with open(csv_path, newline='', encoding='GBK') as csvfile:
        reader = csv.DictReader(csvfile)
        data = list(reader)
    unmatched_persons = []
    for person in persons:
        matched = False
        for row in data:
            if (row['姓名'].strip() == person['name'] and
                row['手机号'].strip() == person['tel'] and
                row['邮箱'].strip() == person['mail']):
                matched = True
                break
        if not matched:
            unmatched_persons.append(person['name'])
    
    return unmatched_persons

# 使用示例
csv_path = 'information.csv'  # 替换为你的CSV文件路径

unmatched = find_person_in_csv(csv_path, persons)
if unmatched:
    print("以下人员的信息在 CSV 文件中不完全符合:")
    for name in unmatched:
        print(name)
else:
    print("所有人员的信息都完全符合。")

[{'name': '蒋献周', 'tel': '13548010827', 'mail': 'jiangxianzhou.cicdi@chinaccs.cn'}, {'name': '齐天宇', 'tel': '17805101835', 'mail': 'qitianyu2007@126.com'}]
以下人员的信息在 CSV 文件中不完全符合:
蒋献周
齐天宇


In [19]:
# 目录的字体、字号、行距与正文相同
import xml.etree.ElementTree as ET

def find_catalog_and_check_format(xml_file):
    # 解析 XML 文件
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # 定义命名空间
    namespaces = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}

    # 查找所有段落
    for paragraph in root.findall('.//w:p', namespaces):
        texts = paragraph.findall('.//w:t', namespaces)
        if texts:
            combined_text = ''.join([t.text for t in texts])
            if '目' in combined_text and '录' in combined_text:
                print(f'Found "目录": {combined_text}')
                # 检测字号和行距
                rPr = paragraph.find('.//w:rPr', namespaces)
                if rPr is not None:
                    rFonts = rPr.find('.//w:rFonts', namespaces=namespaces)
                    if rFonts is not None: 
                        font_eastAsia = rFonts.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}eastAsia')  # 获取东亚字体 
                        print(f'目录字体为{font_eastAsia}.') 
                    sz = rPr.find('w:sz', namespaces)
                    spacing = paragraph.find('.//w:spacing', namespaces)
                    if sz is not None and spacing is not None:
                        size = sz.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}val')
                        line_spacing = spacing.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}line')
                        print(f'字体大小: {size}磅, 行距: {int(line_spacing)/240}倍')
                    else:
                        print('No font size or line spacing found.')
                else:
                    print('No text formatting found for this paragraph.')
                break

# 调用函数并传入XML文件路径
find_catalog_and_check_format('./tt_xml/word/document.xml')

Found "目录": 目      录
目录字体为宋体.
字体大小: 24磅, 行距: 1.5倍


In [21]:
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

def check_table_titles(doc_path):
    # 加载文档
    doc = Document(doc_path)
    
    for i, table in enumerate(doc.tables):
        prev_paragraph = None
        for paragraph in doc.paragraphs:
            if table._element == paragraph._element.getnext():
                prev_paragraph = paragraph
                break

        if prev_paragraph:
            prev_paragraph_text = prev_paragraph.text.strip()
            prev_paragraph_alignment = prev_paragraph.alignment

            # 检查表名是否左对齐
            is_left_aligned = prev_paragraph_alignment == WD_PARAGRAPH_ALIGNMENT.LEFT
            print(f"表名: '{prev_paragraph_text}'")
            print(f"是否左对齐: {'是' if is_left_aligned else '否'}")

            # 如果左对齐，继续检查表名是否居中
            if is_left_aligned:
                # 假设表名字从第一个空格后开始
                if ' ' in prev_paragraph_text:
                    title_start = prev_paragraph_text.index(' ') + 1
                else:
                    title_start = len(prev_paragraph_text)
                
                # 计算表名前后字符数量
                before_title = title_start
                after_title = len(prev_paragraph_text) - title_start
                
                # 判断是否居中
                char_ratio = before_title / max(after_title, 1)
                if 0.8 <= char_ratio <= 1.2:
                    print("表名字位于居中位置")
                else:
                    print(f"表名字不居中: 前面有 {before_title} 个字符，后面有 {after_title} 个字符")
        else:
            print("未找到与表格相邻的段落")

# 指定文档路径
doc_path = './test.docx'
check_table_titles(doc_path)


表名: ''
是否左对齐: 否
表名: '设计文件分发表'
是否左对齐: 否
表名: '表2.1-1                   现网基站分布情况表'
是否左对齐: 是
表名字不居中: 前面有 7 个字符，后面有 27 个字符
表名: '表2.2-1	        运营商需求分场景、分网络制式统计表（单位：个）'
是否左对齐: 是
表名字不居中: 前面有 8 个字符，后面有 30 个字符
表名: '表2.2-2	      运营商通信设备及天馈系统安装需求统计表（单位：个）'
是否左对齐: 否
表名: '表2.2-3                  共建共享统计表(单位：个)'
是否左对齐: 否
表名: '表3.1				  通信基站交流第一级浪涌保护器最大通流容量'
是否左对齐: 否
表名: '表4.8                     B级浪涌保护器配线要求'
是否左对齐: 否
表名: '表7.1.1-1                        各制式间的隔离度要求（单位：dB）'
是否左对齐: 否
表名: '表7.1.1-2                           不同通信系统间的垂直隔离度距离要求（单位：米）'
是否左对齐: 否
表名: '表7.1.1-3                         不同通信系统间的水平隔离度距离要求（单位：米）'
是否左对齐: 否
表名: ''
是否左对齐: 否
表名: '表7.1.2-1        公众曝露控制限值'
是否左对齐: 否
表名: '表7.1.3 可豁免的设施（设备）的等效辐射功率'
是否左对齐: 否
表名: '表11.12                     云南省各地州抗震设防烈度'
是否左对齐: 是
表名字不居中: 前面有 7 个字符，后面有 32 个字符
表名: '表12.4.1                            资产列表'
是否左对齐: 是
表名字不居中: 前面有 8 个字符，后面有 31 个字符
表名: '表12.4.2                       脆弱性分析表'
是否左对齐: 否
表名: '表12.4.3                      威胁来源表'
是否左对齐: 是
表名字不居中: 前面有 8 个字符

In [20]:
# 所有正文字体为宋体小四，英文和数字为新罗马，1.5倍行距

from docx.shared import Pt 

def is_chinese(text):
    for char in text:
        if '\u4e00' <= char <= '\u9fff':
            return True
    return False

def check_paragraph_formatting(doc):
    chinese_font = "宋体"
    english_font = "Times New Roman"
    font_size = Pt(12)
    line_spacing = 1.5
    
    # 确认段落的样式，只针对Normal样式进行检测
    for paragraph in doc.paragraphs:
        # 如果段落内容为空或仅包含空格，则跳过
        if not paragraph.text.strip():
            continue
        
        if paragraph.style.name != 'Normal':
            continue
        
        issues = []
        runs = paragraph.runs
        for run in runs:
            text = run.text.strip()
            if not text:
                continue  # 跳过空文本
            
            if is_chinese(text):
                if run.font.name != chinese_font or run.font.size != font_size:
                    issues.append(f"中文字体应为 {chinese_font}，实际为 {run.font.name if run.font.name else '未设置'}，字体大小应为 {font_size.pt}pt，实际为 {run.font.size.pt if run.font.size else '未设置'}pt")
            else:
                if run.font.name != english_font or run.font.size != font_size:
                    issues.append(f"英文/数字字体应为 {english_font}，实际为 {run.font.name if run.font.name else '未设置'}，字体大小应为 {font_size.pt}pt，实际为 {run.font.size.pt if run.font.size else '未设置'}pt")
        
        if paragraph.paragraph_format.line_spacing != line_spacing:
            issues.append(f"行距应为 {line_spacing} 倍，实际为 {paragraph.paragraph_format.line_spacing if paragraph.paragraph_format.line_spacing else '未设置'}")

        if issues:
            print(f"段落内容: {paragraph.text}\n问题: {', '.join(issues)}\n") 
doc = Document("test.docx")
check_paragraph_formatting(doc)

段落内容: 是
问题: 中文字体应为 宋体，实际为 方正大黑简体，字体大小应为 12.0pt，实际为 18.0pt, 行距应为 1.5 倍，实际为 304800

段落内容: 中国铁塔股份有限公司XX分公司
问题: 中文字体应为 宋体，实际为 方正大黑简体，字体大小应为 12.0pt，实际为 22.0pt, 行距应为 1.5 倍，实际为 304800

段落内容: 2024年工程设计项目
问题: 中文字体应为 宋体，实际为 方正大黑简体，字体大小应为 12.0pt，实际为 22.0pt, 行距应为 1.5 倍，实际为 304800

段落内容: 一阶段设计
问题: 中文字体应为 宋体，实际为 方正大黑简体，字体大小应为 12.0pt，实际为 14.0pt, 行距应为 1.5 倍，实际为 304800

段落内容: （第X册|第×分册）
问题: 英文/数字字体应为 Times New Roman，实际为 方正大黑简体，字体大小应为 12.0pt，实际为 14.0pt, 中文字体应为 宋体，实际为 方正大黑简体，字体大小应为 12.0pt，实际为 14.0pt, 英文/数字字体应为 Times New Roman，实际为 方正大黑简体，字体大小应为 12.0pt，实际为 14.0pt, 中文字体应为 宋体，实际为 方正大黑简体，字体大小应为 12.0pt，实际为 14.0pt, 英文/数字字体应为 Times New Roman，实际为 方正大黑简体，字体大小应为 12.0pt，实际为 14.0pt, 行距应为 1.5 倍，实际为 304800

段落内容: XX铁塔2024年X季度
问题: 英文/数字字体应为 Times New Roman，实际为 方正粗宋简体，字体大小应为 12.0pt，实际为 26.0pt, 中文字体应为 宋体，实际为 方正粗宋简体，字体大小应为 12.0pt，实际为 26.0pt, 英文/数字字体应为 Times New Roman，实际为 方正粗宋简体，字体大小应为 12.0pt，实际为 26.0pt, 中文字体应为 宋体，实际为 方正粗宋简体，字体大小应为 12.0pt，实际为 26.0pt, 行距应为 1.5 倍，实际为 355600

段落内容: 设计项目
问题: 中文字体应为 宋体，实际为 方正粗宋简体，字体大小应为 12.0pt，实

In [8]:
from docx import Document

def count_runs(doc_path, target_text):
    """
    计算给定文本段落中的run数量
    :param doc_path: Word 文档路径
    :param target_text: 目标文本
    """
    doc = Document(doc_path)
    
    for paragraph in doc.paragraphs:
        if target_text in paragraph.text:
            runs = paragraph.runs
            print(f"段落内容: '{paragraph.text}'")
            print(f"该段落中有 {len(runs)} 个 run。")
            
            # 输出每个 run 的内容
            for i, run in enumerate(runs):
                print(f"Run {i+1}: '{run.text}'")
            break

# 指定文档路径和目标文本
doc_path = './test.docx'
target_text = '表2.1-1                   现网基站分布情况表'
count_runs(doc_path, target_text)


段落: 表2.1-1                   现网基站分布情况表
整行字符数: 34
'现网基站分布情况表' 前的字符数: 25
'现网基站分布情况表' 后的字符数: 0
----------------------------------------


In [11]:
from docx import Document

def count_runs(doc_path, target_text):
    """
    计算给定文本段落中的run数量
    :param doc_path: Word 文档路径
    :param target_text: 目标文本
    """
    doc = Document(doc_path)
    
    for paragraph in doc.paragraphs:
        if target_text in paragraph.text:
            runs = paragraph.runs
            print(f"段落内容: '{paragraph.text}'")
            print(f"该段落中有 {len(runs)} 个 run。")
            
            # 输出每个 run 的内容
            for i, run in enumerate(runs):
                print(f"Run {i+1}: '{run.text}'")
            break

# 指定文档路径和目标文本
doc_path = './test.docx'
target_text = '表2.1-1                   现网基站分布情况表'
count_runs(doc_path, target_text)


段落内容: '表2.1-1                   现网基站分布情况表'
该段落中有 11 个 run。
Run 1: '表'
Run 2: ''
Run 3: ''
Run 4: ''
Run 5: ''
Run 6: ''
Run 7: '2.1'
Run 8: ''
Run 9: '-'
Run 10: '1                   '
Run 11: '现网基站分布情况表'


In [1]:
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

def check_font(paragraph, expected_font_name='宋体', expected_font_size=10.5):
    """检查段落字体和大小"""
    for run in paragraph.runs:
        font = run.font
        font_name = font.name
        font_size = font.size.pt if font.size else None
        if font_name != expected_font_name or font_size != expected_font_size:
            return False
    return True

def check_header_footer(doc):
    """检查页眉和页脚的齐全性和字体"""
    for section in doc.sections:
        header = section.header
        footer = section.footer

        # 检查页眉
        if not header.is_linked_to_previous:
            for paragraph in header.paragraphs:
                if not check_font(paragraph):
                    print(f"页眉不符合要求: '{paragraph.text}'")
                else:
                    print(f"页眉齐全且符合要求: '{paragraph.text}'")

        # 检查页脚
        if not footer.is_linked_to_previous:
            for paragraph in footer.paragraphs:
                if not check_font(paragraph):
                    print(f"页脚不符合要求: '{paragraph.text}'")
                else:
                    print(f"页脚齐全且符合要求: '{paragraph.text}'")

# def check_page_numbering(doc):
#     """检查封面、扉页和目录页脚页码是否单独排序"""
#     for i, section in enumerate(doc.sections):
#         footer = section.footer
#         for paragraph in footer.paragraphs:
#             if paragraph.alignment == WD_ALIGN_PARAGRAPH.CENTER:
#                 if i == 0:
#                     print(f"封面页脚页码: {paragraph.text}")
#                 elif i == 1:
#                     print(f"扉页页脚页码: {paragraph.text}")
#                 elif i == 2:
#                     print(f"目录页脚页码: {paragraph.text}")
#                 else:
#                     print(f"正文页脚页码: {paragraph.text}")

def main(docx_path):
    doc = Document(docx_path)
    check_header_footer(doc)
    # check_page_numbering(doc)
docx_path = 'tt.docx'
main(docx_path)


页眉不符合要求: 'XX铁塔2024年X季度一阶段设计'


In [ ]:
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

def check_font(paragraph, expected_font_name='宋体', expected_font_size=10.5):
    """检查段落字体和大小"""
    for run in paragraph.runs:
        font = run.font
        font_name = font.name
        font_size = font.size.pt if font.size else None
        if font_name != expected_font_name or font_size != expected_font_size:
            return False
    return True

def check_header_footer(doc):
    """检查页眉和页脚的齐全性和字体"""
    for section in doc.sections:
        header = section.header
        footer = section.footer

        # 检查页眉
        if not header.is_linked_to_previous:
            for paragraph in header.paragraphs:
                if not check_font(paragraph):
                    print(f"页眉不符合要求: '{paragraph.text}'")
                else:
                    print(f"页眉齐全且符合要求: '{paragraph.text}'")

        # 检查页脚
        if not footer.is_linked_to_previous:
            for paragraph in footer.paragraphs:
                if not check_font(paragraph):
                    print(f"页脚不符合要求: '{paragraph.text}'")
                else:
                    print(f"页脚齐全且符合要求: '{paragraph.text}'")

def check_page_numbering(doc):
    """检查封面、扉页和目录页脚页码是否单独排序"""
    for i, section in enumerate(doc.sections):
        footer = section.footer
        for paragraph in footer.paragraphs:
            if paragraph.alignment == WD_ALIGN_PARAGRAPH.CENTER:
                if i == 0:
                    print(f"封面页脚页码: {paragraph.text}")
                elif i == 1:
                    print(f"扉页页脚页码: {paragraph.text}")
                elif i == 2:
                    print(f"目录页脚页码: {paragraph.text}")
                else:
                    print(f"正文页脚页码: {paragraph.text}")

def main(docx_path):
    doc = Document(docx_path)
    check_header_footer(doc)
    check_page_numbering(doc)

if __name__ == "__main__":
    docx_path = 'test.docx'
    main(docx_path)
